In [1]:
reset -fs

In [2]:
# import libraries
import pandas as pd
import numpy as np
import re
import string
import os

#import nltk
#from nltk.stem import PorterStemmer, SnowballStemmer, LancasterStemmer, WordNetLemmatizer
#nltk.download('stopwords')
#nltk.download('wordnet')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import PCA

# Read in data

In [3]:
modern_love_df = pd.read_csv('modern_love_df.csv')

In [4]:
modern_love_df.shape

(1615, 12)

In [5]:
modern_love_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1615 entries, 0 to 1614
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   article_url             1615 non-null   object
 1   article                 1609 non-null   object
 2   article_id              1615 non-null   object
 3   article_headline        1615 non-null   object
 4   article_abstract        1615 non-null   object
 5   article_snippet         1615 non-null   object
 6   article_lead_paragraph  1604 non-null   object
 7   article_pub_date        1615 non-null   object
 8   article_news_desk       1615 non-null   object
 9   article_section         1615 non-null   object
 10  article_type_material   1615 non-null   object
 11  article_word_count      1615 non-null   int64 
dtypes: int64(1), object(11)
memory usage: 151.5+ KB


# Clean data

In [6]:
pd.set_option("display.max_colwidth", 1)

In [7]:
# drop rows with na in article column

modern_love_df = modern_love_df.dropna(subset=['article'])

In [8]:
modern_love_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1609 entries, 0 to 1614
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   article_url             1609 non-null   object
 1   article                 1609 non-null   object
 2   article_id              1609 non-null   object
 3   article_headline        1609 non-null   object
 4   article_abstract        1609 non-null   object
 5   article_snippet         1609 non-null   object
 6   article_lead_paragraph  1599 non-null   object
 7   article_pub_date        1609 non-null   object
 8   article_news_desk       1609 non-null   object
 9   article_section         1609 non-null   object
 10  article_type_material   1609 non-null   object
 11  article_word_count      1609 non-null   int64 
dtypes: int64(1), object(11)
memory usage: 163.4+ KB


In [9]:
# rename article column

modern_love_df = modern_love_df.rename({'article':'essay'}, axis=1)

In [10]:
# rename the rest of the columns by removing 'article_'

modern_love_df.columns = modern_love_df.columns.str.replace('article_',"")

In [11]:
# check for duplicated rows

modern_love_df.duplicated().sum() # there are none!

0

In [12]:
# convert pub_date to datetime and then keep only the date

modern_love_df['pub_date'] = pd.to_datetime(modern_love_df['pub_date']).dt.date

In [13]:
modern_love_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1609 entries, 0 to 1614
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   url             1609 non-null   object
 1   essay           1609 non-null   object
 2   id              1609 non-null   object
 3   headline        1609 non-null   object
 4   abstract        1609 non-null   object
 5   snippet         1609 non-null   object
 6   lead_paragraph  1599 non-null   object
 7   pub_date        1609 non-null   object
 8   news_desk       1609 non-null   object
 9   section         1609 non-null   object
 10  type_material   1609 non-null   object
 11  word_count      1609 non-null   int64 
dtypes: int64(1), object(11)
memory usage: 163.4+ KB


In [14]:
# see how many articles mention the nytimes in the essay, as this text should be removed

len(modern_love_df[modern_love_df.essay.str.contains('nytimes.com|New York Times')])

105

In [15]:
# drop these essays for times' sake (go back to data acquisition stage to remove these paragraphs, time permitting)

modern_love_df_rev = modern_love_df[~modern_love_df.essay.str.contains('nytimes.com|New York Times')]

In [16]:
# see how many articles mention 'This essay is part of a collaborative project with Black History, Continued. We invited readers and renowned writers to respond to the question “What is Black love today?”'

len(modern_love_df_rev[modern_love_df_rev.essay.str.contains('collaborative project')])

8

In [17]:
modern_love_df_rev[modern_love_df_rev.essay.str.contains('collaborative project')]

url  \
2   https://www.nytimes.com/2022/02/11/style/modern-love-gay-couple.html                       
3   https://www.nytimes.com/2022/02/11/style/modern-love-confidence-self-love.html             
4   https://www.nytimes.com/2022/02/11/style/modern-love-black-boyfriend-applications.html     
5   https://www.nytimes.com/2022/02/11/style/modern-love-queer-dating-open-relationship.html   
7   https://www.nytimes.com/2022/02/11/style/modern-love-bond-father.html                      
8   https://www.nytimes.com/2022/02/11/style/modern-love-black-boys.html                       
9   https://www.nytimes.com/2022/02/11/style/modern-love-healer-heartbreak.html                
10  https://www.nytimes.com/2022/02/11/style/modern-love-teeth-smile.html                      

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [18]:
# remove opening sentences from collaborative project with Black History, Continued

sentences = 'This essay is part of a collaborative project with Black History, Continued. We invited readers and renowned writers to respond to the question “What is Black love today?” '
print(len(sentences))
sentences[:172]
collaborative_project = 'collaborative project'
sentences_to_remove = lambda x: x[172:] if collaborative_project in x else x
modern_love_df_rev['essay'] = modern_love_df['essay'].map(sentences_to_remove)

172


<ipython-input-18-b77f0ab32bac>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modern_love_df_rev['essay'] = modern_love_df['essay'].map(sentences_to_remove)


In [19]:
modern_love_df_rev[['essay']].sample(1)

,essay
1103,"Sometimes the mind slips into dark places. On one such day, I was in the kitchen when my mind busied itself thinking about all I hadn’t achieved. Juggling a career and motherhood, I wished I had more time for both. Believing myself to be alone, I muttered, “What am I good at?” Behind me, quite unexpectedly and matter-of-factly, without a moment’s hesitation, my 6-year-old piped up in his precocious way. “But Mom, without you there would be no laughter.” Nate is almost 33 now, living on the opposite coast. Thankfully, laughter still lights our way on dark days. — Nancy Rae"


In [20]:
modern_love_df_rev.shape

(1504, 12)

In [220]:
modern_love_df_rev.head(5)

url  \
0  https://www.nytimes.com/2022/02/25/style/modern-love-seeking-a-father-for-my-child-relationship-optional.html   
2  https://www.nytimes.com/2022/02/11/style/modern-love-gay-couple.html                                            
3  https://www.nytimes.com/2022/02/11/style/modern-love-confidence-self-love.html                                  
4  https://www.nytimes.com/2022/02/11/style/modern-love-black-boyfriend-applications.html                          
5  https://www.nytimes.com/2022/02/11/style/modern-love-queer-dating-open-relationship.html                        

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

# Topic modeling utility

Before creating a baseline model, create a pipeline function that does the following:

- Arguments are the corpus, vectorizer and topic_modeler
- Applies these in order
- Prints the top 15 words for each topic found from training

In [21]:
# def make_topics(corpus, vectorizer, topic_modeler, print_n_words=15):
#     """A very simple pipeline."""
    
#     # Vectorize documents into a document-word matrix.
#     doc_word_vectors = vectorizer.fit_transform(corpus)
    
#     # Fit the topic model.
#     doc_topic_vectors = topic_modeler.fit_transform(doc_word_vectors)
    
#     # Print the topics.
#     vocab = vectorizer.get_feature_names()
#     for idx, topic in enumerate(topic_modeler.components_):
#         # Select the top 15 words in vocab for this topic.
#         top_words = [vocab[i].upper() for i in topic.argsort()[:-print_n_words-1:-1]]
#         print(f"Topic {idx}:\n", ", ".join(top_words), "\n")
    
#     return doc_topic_vectors

# Make a baseline topic model

As priors, use Tfidf vectorizer and NMF topic modeler

In [22]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import NMF

In [23]:
# corpus = modern_love_df_rev['essay']
# vectorizer = TfidfVectorizer()
# topic_modeler = NMF(5, random_state=42, max_iter=1000) # start with 5 topics

# make_topics(corpus, vectorizer, topic_modeler)

Looks like the topics are capturing lots of stop words - time to remove them!

# Topic Model 2 - remove stop words

Use spaCy package for text processing

In [24]:
# import spacy
# nlp = spacy.load('en_core_web_sm')
# #from spacy.language import Language

In [25]:
# docs = list(nlp.pipe(modern_love_df_rev['essay']))
# modern_love_df_rev['spacy_essay'] = docs

In [26]:
# docs_no_sw = [[str(w) for w in doc if not w.is_stop] for doc in docs]
# modern_love_df_rev['essay_no_stopwords'] = docs_no_sw

In [27]:
# corpus = [token for doc in docs_no_sw for token in doc]
# vectorizer = TfidfVectorizer()
# topic_modeler = NMF(5, random_state=42, max_iter=1000)

# make_topics(corpus, vectorizer, topic_modeler)

Things are looking better, but now it's time to remove non-alpha characters.

# Topic Model 3 - remove punctuation and non-alpha characters

In [28]:
# docs_no_sw_alpha = [[str(w) for w in doc if w.is_alpha and \
#               (not w.is_stop and not w.is_punct and not w.like_num)] for doc in docs]
# modern_love_df_rev['essay_alpha'] = docs_no_sw_alpha

In [29]:
# corpus = [token for doc in docs_no_sw_alpha for token in doc]
# vectorizer = TfidfVectorizer()
# topic_modeler = NMF(5, random_state=42, max_iter=1000)

# make_topics(corpus, vectorizer, topic_modeler)

# Topic Model 4 - keep only nouns

In [30]:
# docs_no_sw_alpha_nouns = [[str(w) for w in doc if w.is_alpha and \
#                            (w.pos_ == 'NOUN' or w.pos_ == 'PROPN') and \
#                            (not w.is_stop and not w.is_punct and not w.like_num)] \
#                           for doc in docs]
# modern_love_df_rev['essay_noun'] = docs_no_sw_alpha_nouns

In [31]:
# corpus = [token for doc in docs_no_sw_alpha_nouns for token in doc]
# vectorizer = TfidfVectorizer()
# topic_modeler = NMF(5, random_state=42, max_iter=1000)

# make_topics(corpus, vectorizer, topic_modeler)

# Topic Model 5 - lemmatize tokens

In [32]:
# docs_no_sw_alpha_nouns_lemma = [[w.lemma_.lower() for w in doc if w.is_alpha and \
#                            (w.pos_ == 'NOUN' or w.pos_ == 'PROPN') and \
#                            (not w.is_stop and not w.is_punct and not w.like_num)] \
#                           for doc in docs]
# modern_love_df_rev['essay_nouns_lemma'] = docs_no_sw_alpha_nouns_lemma

In [33]:
# corpus = [token for doc in docs_no_sw_alpha_nouns_lemma for token in doc]
# vectorizer = TfidfVectorizer()
# topic_modeler = NMF(5, random_state=42, max_iter=1000)

# make_topics(corpus, vectorizer, topic_modeler)

# Look at top most common and least common noun tokens to perhaps remove some

In [34]:
# noun_tokens = [token for doc in modern_love_df_rev['essay_lemma'] for token in doc]
# set_tokens = set(noun_tokens)
# word_freq = Counter(noun_tokens)
# word_freq.most_common(50)

In [35]:
# # least common words
# word_freq.most_common()[-50:]

# CorEx

Helpful resources on CorEx
- <a href="https://github.com/gregversteeg/corex_topic/blob/master/corextopic/example/corex_topic_example.ipynb">notebook on how to use it</a>

In [36]:
!pip install corextopic
!pip install networkx

In [37]:
import scipy.sparse as ss

from corextopic import corextopic as ct
from corextopic import vis_topic as vt

## CorEx Baseline model

In [38]:
vectorizer = CountVectorizer(max_features=20000,binary=True)

doc_word = vectorizer.fit_transform(modern_love_df_rev['essay'])
doc_word = ss.csr_matrix(doc_word)

doc_word.shape # n_docs x m_words

(1504, 20000)

In [39]:
# get words that label the columns (needed to extract readable topics and make anchoring easier)

words = list(np.asarray(vectorizer.get_feature_names()))

In [40]:
# n_hidden is the number of topics
# words is the list of words used in the corpus
# *Brian recommends adding docs=df.data to make it easier to check which sentences are in each resulting topic*

topic_model_base = ct.Corex(n_hidden=5, words=words, seed=1)
topic_model_base.fit(doc_word, words=words, docs=modern_love_df_rev['essay'])

In [41]:
# print all topics from the CorEx topic model

topics = topic_model_base.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: if,or,what,all,there,about,not,could,who,been
1: as,couldn,seemed,right,took,later,without,was,sure,less
2: frame,creative,ritual,reserved,village,bond,resolution,beloved,reflected,envy
3: permission,department,worthy,radical,sliding,repeatedly,uncles,gingerly,stability,anticipating
4: universe,therefore,screamed,stays,harvard,reasoned,landing,angles,indication,scolded


In [42]:
topic_model_base.tcs

array([104.32318172,  51.96132843,   5.48535982,   5.06341038,
         3.96704812])

In [43]:
topic_model_base.get_topics()

[[('if', 0.634797243840824, 1.0),
  ('or', 0.5869965046165553, 1.0),
  ('what', 0.5860359844179419, 1.0),
  ('all', 0.5822022030430116, 1.0),
  ('there', 0.5766822923933494, 1.0),
  ('about', 0.5700054395001276, 1.0),
  ('not', 0.5633406494329875, 1.0),
  ('could', 0.556471046902532, 1.0),
  ('who', 0.5540730054110281, 1.0),
  ('been', 0.550871904075741, 1.0)],
 [('as', 0.4242397756052274, 1.0),
  ('couldn', 0.3361854019438551, 1.0),
  ('seemed', 0.32234669728862086, 1.0),
  ('right', 0.31641962646988553, 1.0),
  ('took', 0.3062458081541832, 1.0),
  ('later', 0.2730202592211932, 1.0),
  ('without', 0.2374747526040581, 1.0),
  ('was', 0.22181333537795472, 1.0),
  ('sure', 0.20584728155557808, 1.0),
  ('less', 0.19942654397876863, 1.0)],
 [('frame', 0.021989744275400704, 1.0),
  ('creative', 0.020890907223346498, 1.0),
  ('ritual', 0.01828537804063526, 1.0),
  ('reserved', 0.017455177377376164, 1.0),
  ('village', 0.017412661010089352, 1.0),
  ('bond', 0.01571894886046132, 1.0),
  ('reso

Model selection: CorEx starts its algorithm with a random initialization, and so different runs can result in different topic models. One way of finding a better topic model is to restart the CorEx algorithm several times and take the run that has the highest TC value (i.e. the run that produces topics that are most informative about the documents).

Time to remove some stop words, punctuation, numbers and lemmatize tokens

## CorEx Model 2 - remove stopwords, lemmatize, lowercase, keep only alpha characters

Use spaCy package for text processing

In [94]:
import spacy
nlp = spacy.load('en_core_web_sm')
from spacy.language import Language
#nlp.add_pipe("merge_noun_chunks")
#nlp.add_pipe("merge_entities")

In [95]:
sw_spacy = nlp.Defaults.stop_words

In [58]:
# add some items to sw_spacy
#more_sw = {'t','s','ll','nt'}
#sw_spacy.update(more_sw)

In [56]:
# remove stop words from essay column
# docs_sw = [[word for word in doc.split() if (word not in sw_spacy)] for doc in modern_love_df_rev['essay']]
# modern_love_df_rev['essay_no_sw'] = [' '.join(map(str, l)) for l in docs_sw]

In [96]:
docs = list(nlp.pipe(modern_love_df_rev['essay']))

The below code was inspired by <a href="https://stackoverflow.com/questions/25692293/inserting-a-link-to-a-webpage-in-an-ipython-notebook">this stackoverflow post</a>

In [120]:
docs_clean = [[token.lemma_.lower() for token in doc if token.is_alpha and \
                        (not token.is_stop and not token.is_punct and not token.like_num and not token.is_digit)] \
                         for doc in docs]
modern_love_df_rev['essay_rev'] = [' '.join(map(str, l)) for l in docs_clean]

<ipython-input-120-0061f97acdcb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modern_love_df_rev['essay_rev'] = [' '.join(map(str, l)) for l in docs_clean]


In [59]:
# old code

# docs_sw_alpha_lemma = [[token.lemma_.lower() for token in doc if \
#                         (not token.is_punct and not token.is_stop and not token.like_num and not token.is_digit)] \
#                          for doc in docs]
# modern_love_df_rev['essay_lemma'] = [' '.join(map(str, l)) for l in docs_sw_alpha_lemma]

In [99]:
vectorizer = CountVectorizer(max_features=20000, binary=True)

doc_word = vectorizer.fit_transform(modern_love_df_rev['essay_rev'])
doc_word = ss.csr_matrix(doc_word)

doc_word.shape # n_docs x m_words

(1504, 20000)

In [100]:
# get words that label the columns (needed to extract readable topics and make anchoring easier)

words = list(np.asarray(vectorizer.get_feature_names()))

In [101]:
topic_model_2 = ct.Corex(n_hidden=5, words=words, seed=1)
topic_model_2.fit(doc_word, words=words, docs=modern_love_df_rev['essay_rev'])

In [102]:
topics = topic_model_2.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: time,go,know,way,come,feel,long,thing,life,want
1: week,voice,late,show,apartment,heart,pull,table,wear,feeling
2: response,reflect,dating,curiosity,reminder,tough,ritual,creative,exclaim,responsible
3: permission,department,tangle,scatter,overly,radical,drunken,sly,doorman,gingerly
4: harvard,extract,assurance,deprive,ripple,sling,animate,command,toddler,khaki


In [170]:
# import tkinter
# import matplotlib
# import matplotlib.pyplot as plt
# matplotlib.use('TkAgg')

# plt.figure(figsize=(10,5))
# plt.bar(range(topic_model_2.tcs.shape[0]), topic_model_2.tcs, color='#4e79a7', width=0.5)
# plt.xlabel('Topic', fontsize=16)
# plt.ylabel('Total Correlation (nats)', fontsize=16)
# plt.show()

In [103]:
print(topic_model_2.tcs)

[72.09614677 29.68482836  5.35299031  5.3291977   2.88328306]


In [104]:
print(topic_model_2.tc)

115.34644620632521


# CorEx Model 3 - try 50 topics

In [105]:
topic_model_3 = ct.Corex(n_hidden=50, words=words, seed=1)
topic_model_3.fit(doc_word, words=words, docs=modern_love_df_rev['essay_rev'])

In [106]:
topics = topic_model_3.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: feel,life,tell,look,ask,say,get,start,right,end
1: time,go,know,way,come,long,thing,want,find,take
2: friend,turn,place,face,room,eye,meet,hour,fall,follow
3: spend,hope,drive,guy,sleep,hair,arm,apartment,wear,girl
4: buy,past,inside,trip,near,eat,throw,window,list,bag
5: maid,mountains,lobe,enlarge,lavender,aboard,subaru,frenzy,historically,muslims
6: fetch,humiliation,victorian,determination,occupy,bonus,cheerful,clerk,minimum,corridor
7: letter,summer,weekly,forget,barely,dead,beach,press,memory,california
8: body,hospital,lift,health,skin,leg,center,road,plastic,ground
9: mentally,inadequate,caretaker,magnificent,materialize,akin,bucket,rapid,listing,womanhood
10: announce,cool,kitchen,fire,provide,tall,blow,feature,recent,cup
11: instill,paul,banquet,twilight,precision,humility,genius,agonize,margarita,headlight
12: middle,blue,low,emotional,game,pretend,shop,effort,gaze,passion
13: mail,result,test,hardly,schedule,beginning,following,page,potential,desk
14: button,sky,awake,bl

In [107]:
topic_model_3.tcs

array([21.66660865, 19.71951039, 13.01940333,  8.15700562,  4.73020545,
        2.67378916,  2.58123697,  2.49280951,  2.16700252,  2.00251426,
        1.93698357,  1.90151179,  1.87179346,  1.86121293,  1.53661956,
        1.51532832,  1.45888423,  1.45664714,  1.38257409,  1.22629575,
        1.22431626,  1.21175397,  1.20642376,  1.12535111,  1.12178565,
        1.10224   ,  1.05005436,  1.04669082,  0.9933184 ,  0.9359119 ,
        0.91613265,  0.89190696,  0.87061242,  0.85986809,  0.83936464,
        0.81429638,  0.81395266,  0.81170822,  0.775956  ,  0.76928301,
        0.69484187,  0.69176532,  0.68409237,  0.66977718,  0.61906472,
        0.60684336,  0.6057628 ,  0.6056417 ,  0.57675788,  0.52578447])

# CorEx Model 4 - try 2 topics

In [108]:
topic_model_4 = ct.Corex(n_hidden=2, words=words, seed=1)
topic_model_4.fit(doc_word, words=words, docs=modern_love_df_rev['essay_rev'])

In [109]:
topics = topic_model_4.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: time,go,know,way,come,feel,long,thing,life,want
1: chore,proudly,jew,acknowledgment,disapproval,judy,orthodox,wrapping,debit,spiky


In [79]:
topic_model_4.tc

115.02884308608039

# CorEx Model 5 - try 10 topics and max_df=.8 in CountVectorizer

In [110]:
vectorizer = CountVectorizer(max_features=20000, binary=True, max_df=0.8)

doc_word = vectorizer.fit_transform(modern_love_df_rev['essay_rev'])
doc_word = ss.csr_matrix(doc_word)

print(doc_word.shape)

words = list(np.asarray(vectorizer.get_feature_names()))

(1504, 20000)


In [111]:
topic_model_5 = ct.Corex(n_hidden=10, words=words, seed=1)
topic_model_5.fit(doc_word, words=words, docs=modern_love_df_rev['essay_rev'])

In [112]:
topics = topic_model_5.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: know,want,take,tell,think,good,day,friend,work,leave
1: time,go,way,come,feel,long,thing,life,find,look
2: later,hand,meet,move,imagine,bed,book,visit,car,hair
3: pretend,club,song,gentle,adorable,shy,garden,exit,comedy,burden
4: retrospect,planning,instantly,admission,satisfy,james,fixate,thump,offend,tentatively
5: acknowledge,commit,shout,forth,pink,grateful,mystery,crave,side,stone
6: mate,salt,unusual,medium,fundamental,stalk,fiction,caution,hover,juice
7: fond,bake,plaster,recount,tire,iron,nighttime,balm,whiz,detective
8: rational,crooked,immigrant,commentary,empathetic,traumatize,ownership,affirm,virginity,dissolution
9: frank,hood,urgent,designer,populate,endeavor,download,thankful,tinder,unpleasant


In [113]:
print(topic_model_5.tcs)
print(topic_model_5.tc)

[35.75276695 29.89888858 29.8185573   5.52932766  4.68164605  4.22471877
  3.41618501  2.74953174  2.60118349  1.78000341]
120.45280895369599


In [86]:
# Let's check out topic : graphics
# topic_model_5.get_top_docs(topic=0, n_docs=10, sort_by='log_prob')

# CorEx Model 6 - 10 topics and remove more stopwords

In [142]:
tokens_to_remove = {'know','want','take','tell','think','thing','ask'}
sw_spacy.update(tokens_to_remove)

In [143]:
docs_2 = list(nlp.pipe(modern_love_df_rev['essay_rev']))

In [144]:
docs_2_clean = [[token for token in doc if token.text not in sw_spacy] for doc in docs_2]
modern_love_df_rev['essay_rev_2'] = [' '.join(map(str, l)) for l in docs_2_clean]

<ipython-input-144-d65b7ab55ec4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modern_love_df_rev['essay_rev_2'] = [' '.join(map(str, l)) for l in docs_2_clean]


In [145]:
vectorizer = CountVectorizer(max_features=20000, binary=True, max_df=0.8)

doc_word = vectorizer.fit_transform(modern_love_df_rev['essay_rev_2'])
doc_word = ss.csr_matrix(doc_word)

print(doc_word.shape)

words = list(np.asarray(vectorizer.get_feature_names()))

(1504, 20000)


In [146]:
topic_model_6 = ct.Corex(n_hidden=10, words=words, seed=1)
topic_model_6.fit(doc_word, words=words, docs=modern_love_df_rev['essay_rev_2'])

In [147]:
topics = topic_model_6.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: good,day,friend,leave,start,right,away,week,let,place
1: time,way,come,feel,long,life,find,look,try,work
2: morning,hair,black,white,ready,party,carry,picture,tear,drop
3: bedside,tackle,impossibly,shade,scott,wing,nursing,sushi,officially,shiny
4: warn,co,financial,palm,butter,treatment,traditional,bond,national,spark
5: recent,exit,garden,bubble,pillow,successful,adorable,russian,backpack,weary
6: retrospect,planning,instantly,backward,satisfy,fragile,james,shyly,bust,fixate
7: fundamental,library,salt,planet,complex,driveway,stalk,hover,monogamous,participate
8: lower,pleasant,affirm,hop,rational,battery,skype,de,commentary,traumatize
9: beam,cooler,tire,nighttime,georgia,jet,fur,interstate,whiz,bungalow


In [148]:
print(topic_model_6.tcs)
print(topic_model_6.tc)

[38.8977179  33.14996079 19.309669    4.57012929  4.56540084  3.5211443
  3.45288227  2.99481697  2.57343416  1.60686393]
114.6420194489523


# CorEx Model 7 - 10 topics and max_df=.8 and min_df=.01 in CountVectorizer

In [149]:
vectorizer = CountVectorizer(max_features=20000, binary=True, max_df=0.8, min_df=.01)

doc_word = vectorizer.fit_transform(modern_love_df_rev['essay_rev_2'])
doc_word = ss.csr_matrix(doc_word)

print(doc_word.shape)

words = list(np.asarray(vectorizer.get_feature_names()))

(1504, 3525)


In [150]:
topic_model_7 = ct.Corex(n_hidden=10, words=words, seed=1)
topic_model_7.fit(doc_word, words=words, docs=modern_love_df_rev['essay_rev_2'])

In [151]:
topics = topic_model_7.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: leave,end,week,let,walk,lose,ago,book,laugh,car
1: soon,family,job,run,drive,pay,mother,pick,food,choice
2: work,turn,right,night,hear,little,face,hold,head,hour
3: way,come,feel,long,life,try,people,need,man,live
4: day,start,away,month,hand,later,sit,early,bad,stop
5: time,find,look,talk,new,mean,kind,love,room,maybe
6: good,place,sense,experience,visit,hair,yes,black,city,fly
7: relationship,fact,wonder,phone,world,couple,past,easy,heart,feeling
8: friend,begin,story,lot,play,break,conversation,dinner,school,share
9: home,stay,watch,eat,near,free,simple,kitchen,color,low


In [152]:
print(topic_model_7.tcs)
print(topic_model_7.tc)

[13.88557833 10.54385396  9.22036791  8.83218696  8.44710073  8.33023918
  7.54114482  5.50298439  4.56745658  2.7632552 ]
79.63416805810043


# CorEx Model 8 - 9 topics and max_df=.7 and min_df=.005 in CountVectorizer

In [206]:
vectorizer = CountVectorizer(max_features=20000, binary=True, max_df=0.7, min_df=.005)

doc_word = vectorizer.fit_transform(modern_love_df_rev['essay_rev_2'])
doc_word = ss.csr_matrix(doc_word)

print(doc_word.shape)

words = list(np.asarray(vectorizer.get_feature_names()))

(1504, 5801)


In [207]:
topic_model_8 = ct.Corex(n_hidden=9, words=words, seed=1)
topic_model_8.fit(doc_word, words=words, docs=modern_love_df_rev['essay_rev_2'])

In [204]:
topics = topic_model_8.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: night,hand,hear,sit,little,hold,soon,point,stand,stop
1: start,right,let,early,spend,bad,hour,fact,stay,realize
2: try,need,man,away,live,month,woman,relationship,home,happen
3: way,long,find,good,people,talk,new,mean,room,maybe
4: work,end,turn,couple,party,large,clean,company,require,emotional
5: friend,begin,hard,change,word,lose,conversation,world,actually,imagine
6: message,online,profile,art,encounter,pile,pocket,wander,crush,attract
7: leave,week,walk,car,heart,arrive,simply,smile,kiss,red
8: place,share,lie,city,include,boyfriend,travel,restaurant,choice,dream


In [205]:
print(topic_model_8.tcs)
print(topic_model_8.tc)

[26.58084139 12.85158589 10.79440924  8.91650956  7.96836379  5.57360239
  4.30435898  3.26692924  2.99636494]
83.2529654150799


# CorEx Model 9 - 10 topics and TF-IDF

In [209]:
vectorizer = TfidfVectorizer(max_features=20000)

doc_word = vectorizer.fit_transform(modern_love_df_rev['essay_rev_2'])
doc_word = ss.csr_matrix(doc_word)

print(doc_word.shape)

words = list(np.asarray(vectorizer.get_feature_names()))

(1504, 20000)


In [210]:
topic_model_9 = ct.Corex(n_hidden=10, words=words, seed=1)
topic_model_9.fit(doc_word, words=words, docs=modern_love_df_rev['essay_rev_2'])

In [211]:
topics = topic_model_9.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: feel,time,way,man,woman,come,relationship,life,friend,talk
1: bouchara,seder,cutlet,plaintiff,tayo,carlito,railway,akari,venus,munich
2: azim,mahjong,hammock,gam,frieda,plushie,melina,preston,chewie,ugonna
3: nicole,chantal,myanmar,brandan,demouth,entidhar,burmese,shelly,freincle,alba
4: greta,isabelle,kerr,storefront,heartless,est,nar,intersect,marjorie,bunnie
5: hertwig,chaulk,qué,vida,pasó,mi,aarp,polyamorous,pollen,politic
6: abdomen,predictability,preconceive,precocious,prayers,powerless,pouch,potted,pot,postmenopausal
7: abaldo,planned,plank,pitifully,piper,pinto,pimple,pigtail,pierce,photo
8: abhorrent,postcard,postabortion,possessed,positively,posit,portuguese,portly,portlandia,portion
9: abnormality,projectile,progeny,procession,probable,prioritize,principal,prince,primitive,presumably


# CorEx Model 10 - 10 topics and TF-IDF and max_df = .7 and min_df= .01

In [214]:
vectorizer = TfidfVectorizer(max_features=20000, max_df=.7, min_df=.01)

doc_word = vectorizer.fit_transform(modern_love_df_rev['essay_rev_2'])
doc_word = ss.csr_matrix(doc_word)

print(doc_word.shape)

words = list(np.asarray(vectorizer.get_feature_names()))

(1504, 3524)


In [217]:
topic_model_10 = ct.Corex(n_hidden=5, words=words, seed=1)
topic_model_10.fit(doc_word, words=words, docs=modern_love_df_rev['essay_rev_2'])

In [218]:
topics = topic_model_10.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: feel,time,way,man,woman,relationship,come,life,friend,long
1: abide,refuge,recovery,recover,recite,react,rain,refugee,queer,quarantine
2: adore,prime,possess,platform,plant,pillow,pie,privacy,photographer,peanut
3: absent,piano,photo,paycheck,patron,patience,pat,pipe,paperwork,oxygen
4: accident,punk,pub,psychologist,proudly,protest,prop,proof,purse,professor


# CorEx Model 11 - 10 topics and CountVectorizer and max_df = .7 and min_df= .01 and n-grams = 2

In [261]:
vectorizer = CountVectorizer(max_features=20000, max_df=.7, min_df=.01, ngram_range=(1,3))

doc_word = vectorizer.fit_transform(modern_love_df_rev['essay_rev_2'])
doc_word = ss.csr_matrix(doc_word)

print(doc_word.shape)

words = list(np.asarray(vectorizer.get_feature_names()))

(1504, 4165)


In [262]:
topic_model_11 = ct.Corex(n_hidden=10, words=words, seed=1)
topic_model_11.fit(doc_word, words=words, docs=modern_love_df_rev['essay_rev_2'])

In [244]:
pd.DataFrame(topic_model_11.p_y_given_x) # n_docs x k_topics

,0,1,2,3,4,5,6,7,8,9
0,0.999999,0.999999,0.999999,0.999999,0.999999,0.999999,0.999999,0.999999,0.000001,0.999999
1,0.999999,0.999999,0.999999,0.999999,0.999999,0.999999,0.999999,0.999999,0.000001,0.999999
2,0.999999,0.999999,0.999999,0.999999,0.999999,0.999999,0.000001,0.999266,0.000001,0.999999
3,0.999999,0.999999,0.999999,0.999999,0.999999,0.999999,0.999999,0.999999,0.000001,0.999999
4,0.999999,0.999999,0.999999,0.999999,0.999999,0.999999,0.000011,0.999999,0.000001,0.999999
...,...,...,...,...,...,...,...,...,...,...
1499,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000009
1500,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000009,0.000001,0.000001
1501,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001
1502,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000001,0.000003,0.000001,0.000001


In [254]:
pd.DataFrame(topic_model_11.labels) # n_docs x k_topics

,0,1,2,3,4,5,6,7,8,9
0,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,False,True,True
3,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...
1499,False,False,False,False,False,False,False,False,False,False
1500,False,False,False,False,False,False,False,False,False,False
1501,False,False,False,False,False,False,False,False,False,False
1502,False,False,False,False,False,False,False,False,False,False


In [267]:
topics = topic_model_11.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: good,friend,begin,meet,wait,story,couple,plan,idea,marry
1: away,place,kind,hand,later,early,family,bad,fact,stay
2: big,car,light,voice,sleep,hair,clear,apartment,arm,black
3: time,long,find,look,try,people,turn,talk,end,new
4: way,come,life,day,need,live,month,relationship,sit,soon
5: work,start,right,week,home,let,night,mean,hear,little
6: window,foot,hospital,living,pound,park,beach,local,road,green
7: leave,head,spend,believe,hour,sure,help,decide,sign,open
8: sign love letter,love letter weekly,weekly email,letter weekly email,letter weekly,sign love,love letter,weekly,modern love,email
9: feel,man,woman,moment,experience,pass,laugh,white,reach,figure


In [264]:
print(topic_model_11.tcs)
print(topic_model_11.tc)

[19.55347721 16.99129584 13.20470215  8.01439473  7.84611342  7.76966188
  5.11406658  4.04109763  3.94603923  3.22754236]
89.70839104454345


In [265]:
topic_model_11.get_top_docs(topic=0)

[('day leave south dakota rex sit talk beneath open hatchback car distance lightning storm open expanse great plains churn sky murky purple talk passionately lithium battery talk common consider interested especially attract person speak care battery volunteer pine ridge reservation build repair infrastructure person greet arrive end long dirt road climb cab skid loader face body warm lyrical version happen storm approach stop talk seriously pleasure body baby date late poem math problem lot line look differ young self envision care date certain time baby love getting marry biological father child maybe admire arrive set criterion alternative sentimental unrealistic especially husband wish list champion year ready able child help sit meditation practice observe worry pregnant discerning love effect fear intensify old trust judgment pressure lot man start smell father decide safe way protect romantic delusion separate story outset try find mate mother time biological constraint easy fig

# CorEx Model 12 - 10 topics and CountVectorizer and max_df = .7 and min_df= .01 and n-grams = 2 and one anchored topic

In [270]:
# anchor one word to the first topic
anchor_words = ['marry']

# anchor the word 'marry' to the first topic
topic_model_12 = ct.Corex(n_hidden=10, seed=1)
topic_model_12.fit(doc_word, words=words, docs=modern_love_df_rev['essay_rev_2'], \
                   anchors=anchor_words, anchor_strength=6);

In [271]:
topics = topic_model_12.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: marry,life,find,people,start,talk,new,woman,begin,maybe
1: try,hard,bad,sign,wonder,write,run,young,ago,line
2: way,friend,home,room,family,stand,watch,change,close,child
3: letter,sign love letter,letter weekly,love letter weekly,letter weekly email,weekly email,sign love,love letter,weekly,email
4: away,hand,walk,course,imagine,drive,small,fear,sleep,easy
5: leave,need,right,week,hear,little,head,early,believe,sure
6: window,foot,baby,hospital,slow,road,tiny,park,brother,bright
7: look,day,end,month,place,later,kind,sit,face,hold
8: good,work,turn,let,night,spend,fact,point,hour,stay
9: time,come,feel,long,man,live,relationship,happen,mean,finally


# CorEx Model 13 - remove more stopwords

In [336]:
more_tokens_to_remove = {'man','woman','person','people','relationship','feel'}
sw_spacy.update(more_tokens_to_remove)

In [337]:
docs_2_clean_2 = [[token for token in doc if token.text not in sw_spacy] for doc in docs_2]
modern_love_df_rev['essay_rev_3'] = [' '.join(map(str, l)) for l in docs_2_clean_2]

<ipython-input-337-8b3d8bea7125>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modern_love_df_rev['essay_rev_3'] = [' '.join(map(str, l)) for l in docs_2_clean_2]


In [371]:
vectorizer = CountVectorizer(max_features=20000, binary=True, max_df=.8, min_df=.005)

doc_word = vectorizer.fit_transform(modern_love_df_rev['essay_rev_3'])
doc_word = ss.csr_matrix(doc_word)

print(doc_word.shape)

words = list(np.asarray(vectorizer.get_feature_names()))

(1504, 5796)


In [390]:
topic_model_13 = ct.Corex(n_hidden=5, seed=1)
topic_model_13.fit(doc_word, words=words, docs=modern_love_df_rev['essay_rev_3'], \
                   anchors=anchor_words, anchor_strength=2);

In [391]:
topics = topic_model_13.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: time,way,come,long,life,find,look,good,try,friend
1: day,live,week,place,hand,hear,sit,head,hold,year
2: buy,table,mail,decade,town,graduate,blue,support,form,view
3: home,night,hard,family,instead,word,lose,course,explain,high
4: finally,marriage,send,husband,wife,expect,message,nice,term,sex


In [392]:
print(topic_model_13.tcs)
print(topic_model_13.tc)

[27.89178675 16.00656831 17.3941484  15.38456016 16.98599933]
93.6630629430795


# CorEx Model 14 - add more anchors

In [387]:
anchor_words = ['hospital']

topic_model_14 = ct.Corex(n_hidden=8, seed=1)
topic_model_14.fit(doc_word, words=words, docs=modern_love_df_rev['essay_rev_3'], \
                   anchors=anchor_words, anchor_strength=6);

In [388]:
topics = topic_model_14.get_topics()
for n,topic in enumerate(topics):
    topic_words,_,_ = zip(*topic)
    print('{}: '.format(n) + ','.join(topic_words))

0: time,way,come,long,life,find,look,try,turn,need
1: new,family,old,learn,plan,share,grow,able,marriage,inside
2: home,begin,walk,watch,help,return,understand,door,late,car
3: writer,encounter,accord,tale,account,holiday,crisis,explanation,garden,cute
4: good,leave,work,start,right,away,week,let,night,hear
5: day,friend,place,sit,early,finally,fact,stand,stay,phone
6: room,meet,spend,open,run,school,buy,remember,easy,sound
7: window,daughter,nurse,birth,pregnant,sun,paint,wood,toy,gray


In [389]:
print(topic_model_14.tcs)
print(topic_model_14.tc)

[14.90851715  7.60201598 29.54186906  3.11007085 13.18148948  9.74396495
 11.259201    4.81889542]
94.16602388097101


Create custom factory to add to spacy pipeline to remove entities whose labels are 'PERSON'

In [76]:
#@Language.component('person_removal')
#def person_removal(doc):
#    ents = list(doc.ents)
   # for ent in ents:
   #     if ent.label_ == 'PERSON':
    #        ents.remove(ent)
   # ents = tuple(ents)
  #  doc.ents = ents
   # return (doc)
#Language.component("person_removal", func=person_removal)

In [77]:
#nlp.add_pipe('person_removal')

In [78]:
#print(nlp.pipe_names)

Add some words to the stop words list to be removed from the corpus

In [87]:
sw_spacy = nlp.Defaults.stop_words

# verbs and a noun from top 50 tokens to remove
top_tokens_to_remove = {'relationship', 'say', 'love', 'know', 'like', 'want', 'feel', 'think', 'tell', 'go', 'ask', 'look','come'}
sw_spacy.update(top_tokens_to_remove)

# bottom 20 tokens to remove

twenty_least_common = word_freq.most_common()[-20:]
bottom_tokens_to_remove = set([a_tuple[0] for a_tuple in twenty_least_common])
sw_spacy.update(bottom_tokens_to_remove)

Apply updated stop words list to tokens

In [91]:
docs_clean_2 = [[w for w in doc if w not in sw_spacy] for doc in docs_clean]
modern_love_df_rev['essay_fewer_stopwords'] = docs_clean_2

<ipython-input-91-d9a1a52d80ab>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  modern_love_df_rev['essay_fewer_stopwords'] = docs_clean_2


In [93]:
modern_love_df_rev[['essay_clean','essay_fewer_stopwords']].head(2)

,essay_clean,essay_fewer_stopwords
0,"[day, leave, south, dakota, rex, sit, talk, beneath, open, hatchback, car, distance, lightning, storm, move, open, expanse, great, plains, churn, sky, murky, purple, talk, passionately, lithium, battery, talk, common, consider, interested, especially, attract, person, speak, ask, care, battery, volunteer, pine, ridge, reservation, build, repair, infrastructure, person, greet, arrive, end, long, dirt, road, climb, cab, skid, loader, see, face, body, warm, lyrical, version, happen, storm, approach, stop, talk, take, seriously, pleasure, body, want, baby, date, late, like, poem, like, math, problem, lot, line, look, differ, young, self, envision, care, date, certain, time, baby, love, getting, marry, want, like, biological, father, child, ...]","[day, leave, south, dakota, rex, sit, talk, beneath, open, hatchback, car, distance, lightning, storm, open, expanse, great, plains, churn, sky, murky, purple, talk, passionately, lithium, battery, talk, common, consider, interested, especially, attract, person, speak, care, battery, volunteer, pine, ridge, reservation, build, repair, infrastructure, person, greet, arrive, end, long, dirt, road, climb, cab, skid, loader, face, body, warm, lyrical, version, happen, storm, approach, stop, talk, seriously, pleasure, body, baby, date, late, poem, math, problem, lot, line, differ, young, self, envision, care, date, certain, time, baby, getting, marry, biological, father, child, maybe, admire, arrive, set, criterion, alternative, sentimental, unrealistic, especially, husband, wish, ...]"
2,"[peter, hitch, wagon, young, early, wildly, handsome, gay, soulful, brown, eye, head, loose, unruly, curl, raise, african, american, woman, white, preacher, man, peter, spend, childhood, steep, evangelical, ministry, terrified, father, sermon, soothe, mother, inclination, administer, human, aspect, congregation, lead, night, meet, peter, fall, way, love, kismet, laugh, word, time, nuyorican, poets, cafe, premier, debut, joint, performance, poet, try, new, york, city, poetry, scene, read, piece, open, mic, slam, peter, admirer, way, applauding, crowd, tell, simultaneously, embody, fury, father, vulnerability, mother, destine, true, love, ask, fate, fix, say, god, leave, room, surprise, time, meet, fall, peter, parent, long, divorced, father, leave, ...]","[peter, hitch, wagon, young, early, wildly, handsome, gay, soulful, brown, eye, head, loose, unruly, curl, raise, african, american, woman, white, preacher, man, peter, spend, childhood, steep, evangelical, ministry, terrified, father, sermon, soothe, mother, inclination, administer, human, aspect, congregation, lead, night, meet, peter, fall, way, kismet, laugh, word, time, nuyorican, poets, cafe, premier, debut, joint, performance, poet, try, new, york, city, poetry, scene, read, piece, open, mic, slam, peter, admirer, way, applauding, crowd, simultaneously, embody, fury, father, vulnerability, mother, destine, true, fate, fix, god, leave, room, surprise, time, meet, fall, peter, parent, long, divorced, father, leave, church, peter, explore, truth, believe, ...]"


The below cell did run

In [33]:
# tokenization into words and parts of speech tagging

#from nltk.tokenize import word_tokenize, sent_tokenize
#from nltk.tag import pos_tag

#tokens = [pos_tag(word_tokenize(row)) for row in modern_love_df['essay']] # labels each word as a part of speech

The below cell ran but took a while to do so

In [ ]:
# named entity recognition

#from nltk.chunk import ne_chunk

#entities = [ne_chunk(token) for token in tokens] # this extracts entities from the list of words
#entities.draw()

The below cell did run

In [22]:
# Import module
#from nltk.tokenize import RegexpTokenizer
#from nltk.tag import pos_tag

# Create an instance of RegexpTokenizer for alphanumeric tokens
#tokeniser = RegexpTokenizer(r'\w+')

# Tokenize text and tag parts of speech
#tokens = [pos_tag(tokeniser.tokenize(row)) for row in modern_love_df['essay']] # labels each word as a part of speech

The below cell didn't work for me

In [40]:
# function to remove tuples in tokens that contain 'NNP' or 'NNPS'

#def remove_items(lst, item):
    #r = []
    #for i in lst:
       # if isinstance(i, list):
        #    r.append(remove_items(i, item))
       # elif item not in i:
         #   r.append(i)
   # return r

In [41]:
# remove proper nouns from tokenized text
#tokens_rev = remove_items(tokens,'NNP')
#tokens_rev = remove_items(tokens,'NNPS')

The below cell didn't work for me

In [43]:
#def filter_nnp_nnps(x):
  #  return [(u,v) for (u,v) in x if (v !='NNP') | (v != 'NNPS')]

#tokens_rev = [filter_nnp_nnps(sub_list) for sub_list in tokens]

The below cell works

In [30]:
# remove numbers and punctuation in essay column
    # used this resource: https://datagy.io/python-remove-punctuation-from-string/

#alphanumeric = lambda x: re.sub('[^a-zA-Z]', ' ', str(x)) # removes numbers
#punc_lower = lambda x: re.sub(r'[^\w\s]', ' ',str(x).lower()) # removes punctuation and makes all letters lowercase

#modern_love_df_rev['essay_rev'] = modern_love_df_rev.essay.map(alphanumeric).map(punc_lower)

In [31]:
# the below code and text came from here: https://github.com/LucDemortier/MachineLearningWithTweets/blob/master/YogaTweetAnalysis.ipynb
'''
Load NLTK's English stopwords as well as its stemmer, then define:
* tokenize_and_stem: to tokenize into sentences, then words, 
  and return the stems of the words;
* tokenize_only: to only tokenize

Also create a stemmed version of the list of stopwords.  Note that the order 
of operations matters: (filter,stem) is not the same as (stem,filter), where "filter"
means "remove stopwords" in the first case and "remove stemmed stopwords" in the 
second.  For example, the word "willing" is not a stopword, but its stem "will" is.  
Thus, the order (stem,filter) tends to eliminate more tokens than (filter,stem).

Finally, create a dictionary that allows us to go back from stem to word (a one-to-many 
relationship that we arbitrarily reduce to a one-to-one relationship to facilitate 
interpretation of results later.)
'''
#import nltk
#from   nltk.stem.snowball import SnowballStemmer
#from   more_itertools import unique_everseen

#stopwords = nltk.corpus.stopwords.words('english')
#stopwords.append("'ve")
#stopwords.append("'re")
#more_stopwords = ["'d", 'could', 'might', 'must', "n't", 'need', 'r', 'sha', 'v', 'wo', 'would']
#stopwords.extend(more_stopwords)
#print('Stop words (%i): \n%s\n' %(len(stopwords),[str(word) for word in stopwords]))

#stemmer           = SnowballStemmer("english")
#stemmed_stopwords = list(unique_everseen([stemmer.stem(word) for word in stopwords]))
#print('Stemmed stop words (%i): \n%s\n' %(len(stemmed_stopwords),[str(word) for word in stemmed_stopwords]))

#alphanumeric = lambda x: re.sub('[^a-zA-Z]', ' ', str(x)) # removes numbers
#punc_lower = lambda x: re.sub(r'[^\w\s]', ' ',str(x).lower()) # removes punctuation and makes all letters lowercase

#def tokenize_and_stem(text):
    
    #tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
   # stems = [stemmer.stem(t) for t in tokens]
    #return stems

#def tokenize_only(text):

  #  tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
   # return tokens

'\nLoad NLTK\'s English stopwords as well as its stemmer, then define:\n* tokenize_and_stem: to tokenize into sentences, then words, \n  and return the stems of the words;\n* tokenize_only: to only tokenize\n\nAlso create a stemmed version of the list of stopwords.  Note that the order \nof operations matters: (filter,stem) is not the same as (stem,filter), where "filter"\nmeans "remove stopwords" in the first case and "remove stemmed stopwords" in the \nsecond.  For example, the word "willing" is not a stopword, but its stem "will" is.  \nThus, the order (stem,filter) tends to eliminate more tokens than (filter,stem).\n\nFinally, create a dictionary that allows us to go back from stem to word (a one-to-many \nrelationship that we arbitrarily reduce to a one-to-one relationship to facilitate \ninterpretation of results later.)\n'

# Document similiarity: vectorize

In [100]:
# define corpus

docs_list_clean = [' '.join(doc) for doc in docs_clean_2]
corpus = docs_list_clean

In [101]:
'''
Apply tfidf transformation
max_df = maximum fraction of documents containing the word, for the word to be included in tfidf matrix
min_df = minimum fraction ...
ngram_range: We'll look at unigrams, bigrams and trigrams.
'''

vec_tfidf = TfidfVectorizer(max_df=0.80, min_df=0.01, ngram_range=(1,3))
X_tfidf = vec_tfidf.fit_transform(corpus).toarray()
pd.DataFrame(X_tfidf, columns=vec_tfidf.get_feature_names())

,abandon,abide,ability,able,abroad,abruptly,absence,absent,absolutely,absorb,...,young,young brother,young child,young girl,young man,young sister,young woman,youth,zone,zoom
0,0.000000,0.0,0.000000,0.050780,0.0,0.0,0.039439,0.0,0.000000,0.0,...,0.022051,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.040593,0.0,0.037210,0.026029,0.0,0.0,0.000000,0.0,0.043847,0.0,...,0.067819,0.050952,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.058661,0.066108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.038088,0.0,0.034913,0.073267,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1499,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1500,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.125728,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1501,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1502,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Make a baseline topic model

The below cell works

In [18]:
# read in custom stop words
#with open('stop_words_english.txt', 'r') as f:
    #stopwords_custom = [s.strip() for s in f.readlines()]

The below cell works

In [19]:
# create function to lemmatize tokens and eliminate stopwords
#def lemmatize_and_eliminate(text):
    #lemmatizer = WordNetLemmatizer()
    #text = [word for word in text.split() if word not in stopwords_custom]
    #text = [lemmatizer.lemmatize(word) for word in text]
    #return " ".join(text)

The below cell works

In [26]:
# lemmatize and remove stop words from corpus
#corpus = modern_love_df_rev.essay_rev.map(lambda x: lemmatize_and_eliminate(x))

In [29]:
i = 600

doc = (pd.Series(name=corpus[i],
                data=X_tfidf[i], 
                index=vec_tfidf.get_feature_names()) 
        .sort_values(ascending=False))

doc.head()

linda       0.534776
accident    0.197141
motel       0.145908
car         0.120504
bowel       0.119363
Name: thursday morning cooking breakfast pajama daughter madeleine helping transferring egg cardboard carton refrigerator egg box pressing button elevator licking cake spoon morning performing duty sleepy devotion whisk asked madeleine throw egg peered curiously work egg nodded break throw madeleine ll catch looked egg small hand reason began carefully inspect shell trifling culinary request egg mixture unscripted moment trust father daughter wanted trust threw delicate save mother died year earlier sudden undiagnosed disorder suspected case marfan syndrome people heard madeleine talk day private nod listen ll share don answer time subtle exchange occurred week making dinner noticed madeleine leafing fashion magazine evening ve marry choked retrieved magazine opened featuring catwalk model blue dress yellow cape circled blue crayon bit snow white answer wasn time disney princess pla

# Principal Component Analysis

In [ ]:
# 
pca = PCA()